# 1. 环境配置
## 1.1 安装 openai 库

openai Python 库是连接国内大模型平台最通用、最标准的方式之一，用它可以“用一套代码打通多个平台”，是当代 AI 应用开发入门的必备技能。

In [ ]:
! pip install openai==2.11.0 dashscope==1.25.4

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## 1.2 获取大模型 API_Key

### 1.2.1 AI Studio 
在 AI Studio 的 BML 中已经内置了默认的 API_Key, 因此我们无需修改即可进行使用。假如我们希望在本地调用，可以在[我的控制台](https://aistudio.baidu.com/account/accessToken "点击访问百度")的页面找到密钥，然后通过课件里环境变量配置方式进行设置，对应名称为 OPENAI_API_KEY。

In [2]:
import os

# 如未使用环境变量配置 API Key，可取消以下注释并填写你的 Key
# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxx"

### 1.2.2 百炼大模型平台
请先前往[阿里云百炼大模型平台](https://bailian.console.aliyun.com/?tab=model#/api-key)注册账号，然后通过同样的方式设置为 DASHSCOPE_API_KEY 即可。

In [3]:
import os

# 如未使用环境变量配置 API Key，可取消以下注释并填写你的 Key
# os.environ["DASHSCOPE_API_KEY"] = "sk-yyyyyyyy"

## 1.3 获取 AI Studio 模型信息

我们可以通过以下代码来了解 AI Studio 中所支持的所有模型。另外我们也可以通过[百度大模型 API 文档](https://www.baidu.com "点击访问百度")获取更多相关信息。

In [4]:
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # 含有 AI Studio 访问令牌的环境变量，https://aistudio.baidu.com/account/accessToken,
    base_url="https://aistudio.baidu.com/llm/lmapi/v3",  # aistudio 大模型 api 服务域名
)

models = client.models.list()
for model in models.data:
    print(model.id)

embedding-v1
Stable-Diffusion-XL
ernie-3.5-8k
ernie-4.0-8k
ernie-4.0-turbo-8k
ernie-speed-8k
ernie-speed-128k
ernie-tiny-8k
ernie-char-8k
ernie-lite-8k
bge-large-zh
ernie-4.0-turbo-128k
deepseek-r1
deepseek-v3
ernie-4.5-turbo-vl-32k
ernie-4.5-turbo-128k
ernie-4.5-turbo-32k
ernie-x1-turbo-32k
deepseek-r1-250528
ernie-lite-pro-128k
ernie-speed-pro-128k
ernie-4.0-turbo-8k-latest
ernie-4.0-8k-latest
qwq-32b
qwen2.5-vl-7b-instruct
qwen2.5-vl-32b-instruct
qwen2.5-7b-instruct
llama-4-scout-17b-16e-instruct
llama-4-maverick-17b-128e-instruct
qwen3-4b
qwen3-8b
qwen3-32b
qwen3-30b-a3b
qwen3-235b-a22b
ernie-4.5-vl-28b-a3b
ernie-4.5-21b-a3b
ernie-4.5-0.3b
ernie-4.5-turbo-vl-preview
ernie-4.5-turbo-128k-preview
qwen3-coder-30b-a3b-instruct
kimi-k2-instruct
qwen3-coder-480b-a35b-instruct
ernie-4.5-turbo-vl
ernie-x1.1-preview
ernie-4.5-21b-a3b-thinking
ernie-4.5-vl-28b-a3b-thinking
ernie-5.0-thinking-preview


# 2. 模型调用

在使用大模型 API 时，我们通常会遇到两种输出模式：普通输出（非流式） 和 流式输出（Stream）。

## 2.1 非流式输出

特点：
- 一次性返回完整的模型生成结果。
- 代码逻辑简单，适合短文本或对实时性要求不高的场景。

执行流程：
1. 发送请求给模型。
2. 模型在服务器端完成推理，生成完整结果。
3. 一次性返回完整回答，供后续处理。

适用场景：

- 短文本问答
- 批量任务处理
- 对实时性要求不高的情况

In [5]:
import os
from openai import OpenAI

# 1. 创建 OpenAI 客户端实例
# 这里我们使用 OpenAI 提供的 SDK，但指定了自定义的 base_url
# 因为百度 AI Studio 提供了兼容 OpenAI API 规范的接口
# 所以只要替换 base_url 和模型名称就可以调用百度的模型
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # 从系统环境变量中读取 API Key，避免在代码中写死，保证安全
    base_url="https://aistudio.baidu.com/llm/lmapi/v3",  # 指定 API 接口的基础 URL，这里是百度 AI Studio 的接口地址
)

# 2. 调用 Chat Completion 接口，发起一次对话请求
chat_completion = client.chat.completions.create(
    messages=[  # 对话的历史消息，支持多轮对话
        {
            'role': 'system',  # 系统角色，用于设定 AI 助手的身份和行为
            'content': '你是 AI Studio 实训AI开发平台的开发者助理，你精通开发相关的知识，负责给开发者提供搜索帮助建议。'
        },
        {
            'role': 'user',  # 用户角色，表示这是用户输入的内容
            'content': '你好，请介绍一下AI Studio'
        }
    ],
    model="ernie-3.5-8k",  # 指定调用的模型，这里是百度文心大模型的 3.5 版本，支持 8k 上下文
)

# 3. 打印 AI 模型的回复
# choices[0] 表示取第一个回答（有时可能返回多个回答）
# message.content 表示提取回答的具体文本内容
print(chat_completion.choices[0].message.content)


你好！AI Studio是一个集成了AI开发、实训和社区交流功能的综合性平台，它为开发者提供了从学习到实践的一站式服务。以下是关于AI Studio的一些详细介绍：

1. **实训环境**：AI Studio为开发者提供了在线的实训环境，这意味着开发者无需在本地搭建复杂的开发环境，只需通过浏览器即可访问和使用平台提供的计算资源。这对于初学者来说尤其友好，因为他们可以立即开始学习和实践，而无需担心环境配置的问题。
2. **丰富的课程资源**：平台上有大量的AI相关课程，涵盖了从基础知识到高级应用的各个方面。这些课程由经验丰富的讲师授课，内容详实且易于理解。无论是初学者还是有一定经验的开发者，都能在这里找到适合自己的学习路径。
3. **实战项目**：除了课程学习，AI Studio还提供了丰富的实战项目供开发者挑战。这些项目通常与实际应用场景紧密相关，能够帮助开发者将所学知识应用到实际中，提升他们的实战能力。
4. **社区交流**：AI Studio还拥有一个活跃的开发者社区，开发者可以在这里交流心得、分享经验、解决问题。社区中的讨论和互动不仅能够帮助开发者更快地成长，还能激发他们的创新思维和合作精神。
5. **竞赛活动**：平台会定期举办各种AI相关的竞赛活动，鼓励开发者积极参与并展示自己的才华。这些竞赛不仅提供了丰厚的奖品和荣誉，还能为开发者提供一个展示自己作品的舞台，增加他们的曝光度和影响力。
6. **技术支持**：对于在使用过程中遇到问题的开发者，AI Studio提供了专业的技术支持团队。他们能够及时响应开发者的需求，提供有效的解决方案和技术指导。

对于想要深入学习和实践AI开发的开发者来说，AI Studio无疑是一个不可多得的好平台。它不仅提供了丰富的学习资源和实战机会，还拥有一个活跃的社区和专业的技术支持团队，能够帮助开发者更快地成长和进步。如果你对AI开发感兴趣，不妨来AI Studio看看吧！


## 2.2 流式输出（Stream）

特点：

- 模型生成内容的同时分块（chunk）返回。
- 用户可以像“实时打字”一样看到逐步生成的内容。
- 适合长文本或实时交互场景。

执行流程：

1. 开启 stream=True 参数。
2. 模型生成内容时分多次返回，每次返回一个数据块。
3. 客户端逐块处理输出，实时展示。

适用场景：

- 长文本生成（如文章、代码）
- 实时交互（如聊天机器人）
- 对用户体验要求高的前端应用

In [6]:
import os
from openai import OpenAI

# 1. 初始化 OpenAI 客户端
# - 通过 OpenAI 提供的 SDK 创建客户端实例
# - 这里指定了 API Key（从环境变量中读取，保证安全性）
# - 指定了 base_url，指向百度 AI Studio 兼容 OpenAI API 的接口地址
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # 从系统环境变量中读取 API Key，建议在系统中提前设置
    base_url="https://aistudio.baidu.com/llm/lmapi/v3",  # 百度 AI Studio 提供的 API 入口
)

# 2. 创建一个 **流式对话（stream=True）**
# - 这里调用 chat.completions.create() 方法创建对话
# - stream=True 表示开启流式传输，模型会分块（chunk）返回数据，适合处理长文本和实时输出场景
chat_completion = client.chat.completions.create(
    model="ernie-3.5-8k",  # 指定使用的模型，这里是文心一言 3.5 模型，支持 8k token 上下文
    messages=[  # 定义对话历史，支持多轮对话
        {
            "role": "system",  # 系统角色：用于设定 AI 的身份、知识领域或行为风格
            "content": "你是 AI Studio 实训AI开发平台的开发者助理，你精通开发相关的知识，负责给开发者提供搜索帮助建议。"
        },
        {
            "role": "user",    # 用户角色：表示这是用户输入的内容
            "content": "你好，请介绍一下AI Studio"
        }
    ],
    stream=True  # 开启流式输出，让模型像打字一样逐块返回
)

# 3. 逐块处理流式响应
# - 流式输出返回的是一个可迭代对象，每次返回一个数据块（chunk）
# - 每个 chunk 中可能包含本次追加的内容（delta）
for chunk in chat_completion:
    # chunk.choices：返回的候选回答列表
    # chunk.choices[0].delta.content：本次追加的文本内容（可能为空，需要判断）
    if chunk.choices and chunk.choices[0].delta.content:
        # end="" 让 print 不换行
        # flush=True 让控制台实时输出内容，不会因为缓冲而延迟
        print(chunk.choices[0].delta.content, end="", flush=True)

# 4. 在流式输出完成后，统一换行，避免最后一行和后续输出粘在一起
print()


您好，很高兴为您介绍AI Studio！AI Studio是一个集成了丰富AI开发资源和工具的实训平台，它为开发者提供了一个便捷、高效的开发环境，旨在帮助开发者快速上手并深入实践人工智能开发。以下是关于AI Studio的详细介绍：

**一、核心功能**

1. **一站式开发环境**：AI Studio提供了在线的集成开发环境（IDE），支持多种编程语言和框架，开发者无需在本地配置复杂的开发环境，即可直接进行AI项目的开发、训练和部署。

2. **丰富的教程与课程**：平台内置了大量的AI开发教程和实战课程，涵盖了从基础知识到高级应用的各个方面，适合不同水平的开发者学习提升。

3. **预训练模型与数据集**：AI Studio集成了众多预训练模型和公开数据集，开发者可以轻松调用这些资源，加速项目的开发进程。

4. **社区交流与分享**：平台拥有活跃的开发者社区，开发者可以在这里交流心得、分享经验、解决问题，形成良好的学习氛围。

**二、特色优势**

1. **易用性**：AI Studio的界面简洁友好，操作便捷，即使对于初学者也能快速上手。

2. **高效性**：通过集成化的开发环境和丰富的资源，开发者可以更加高效地完成AI项目的开发。

3. **实践性**：平台注重实战应用，提供了大量的实战项目和案例，帮助开发者将理论知识转化为实践能力。

4. **开放性**：AI Studio支持开源和分享，开发者可以将自己的项目和成果发布到平台上，与更多人交流和合作。

**三、适用场景**

AI Studio适用于多种AI开发场景，包括但不限于：

- **学术研究**：学者和研究人员可以利用平台上的资源和工具进行AI领域的研究和探索。
- **企业开发**：企业开发者可以在平台上快速搭建和部署AI应用，提升业务效率和竞争力。
- **个人学习**：对于对AI感兴趣的初学者和爱好者，AI Studio提供了丰富的学习资源和实战项目，帮助他们快速入门并提升技能。

**四、使用建议**

如果您是AI开发的初学者，建议您从平台上的基础教程和课程开始学习，逐步掌握AI开发的基本知识和技能。同时，您可以积极参与社区交流，与其他开发者分享经验和心得，共同进步。如果您已经有一定的AI开发经验，那么您可以利用平台上的高级资源和工具进行更加深入的研究和实践，探

## 2.3 切换模型

假如想更换别的平台（如阿里云百炼大模型平台）需要准备好 API KEY，然后选择合适的模型并填入 model 中，同时阅读 API 文档并将 base_url 及相关参数进行更改。


In [7]:
import os
from openai import OpenAI

# os.environ["DASHSCOPE_API_KEY"] = "你的 API_KEY"

client = OpenAI(
   api_key=os.environ.get("DASHSCOPE_API_KEY"), 
   base_url="https://dashscope.aliyuncs.com/compatible-mode/v1", 
)

chat_completion = client.chat.completions.create(
  messages=[
    {'role': 'system', 'content': '你是百炼大模型平台的开发者助理，你精通开发相关的知识，负责给开发者提供搜索帮助建议。'},
    {'role': 'user', 'content': '你好，请介绍一下百炼大模型平台'}
  ],
  model="qwen-max",
)

print(chat_completion.choices[0].message.content) # 打印输出

你好！百炼大模型平台是一个专注于为企业和个人开发者提供高质量AI模型训练、优化及部署服务的云平台。它集合了多种先进的人工智能技术，包括但不限于自然语言处理(NLP)、计算机视觉(CV)等领域的最新研究成果，旨在通过强大的计算资源和易用的开发工具，帮助用户快速构建适合自己应用场景的人工智能解决方案。

### 主要特点：

1. **高效灵活的模型训练**：支持使用预置或自定义的数据集进行模型训练，并提供了丰富的参数配置选项来满足不同场景下的需求。
2. **自动化的模型优化与压缩**：采用先进的算法对训练完成后的模型进行自动优化，以减少模型体积同时保持较高的准确率，便于在边缘设备上运行。
3. **便捷的一键式部署**：无论是公有云还是私有服务器，都可以轻松实现模型的一键部署，大大降低了将AI能力集成到现有系统中的门槛。
4. **开放兼容性强**：支持主流框架如TensorFlow, PyTorch等，同时也鼓励社区贡献开源项目，促进技术交流与发展。
5. **安全保障**：重视数据安全与隐私保护，在整个过程中采取了严格的安全措施，确保用户信息不被泄露。

### 适用对象：
- 对于希望利用AI提升业务效率的企业来说，百炼大模型平台能够提供从零开始建立AI应用所需的一切；
- 对研究者而言，则是一个理想的实验环境，可以用来测试新想法并迅速迭代改进；
- 而对于个人爱好者或者小团队来讲，该平台也十分友好，即使没有深厚的技术背景也能轻松上手尝试。

总之，无论你是想要深入研究AI技术的专业人士，还是仅仅想为自己的项目添加一些智能化元素的普通开发者，百炼大模型平台都能为你提供强有力的支持。希望这对你有所帮助！如果还有其他问题或需要更详细的信息，请随时告诉我。


# 3. 提示词工程



## 3.1 系统角色
一般而言，系统角色分为以下三类：
- user（用户）：表示由用户发出的问题或指令，即模型的输入。这个角色的内容是模型主要关注和响应的部分。（日常和AI的对话就是user）
- system（系统）：用于设定整个对话的行为规范，例如定义模型的身份、语气或任务方向。通常只设置一次，放在对话最前面。
- assistant（照顾使用）：表示由模型生成的回复，用于模拟助手的回答，通常跟在 user 之后。

### 3.1.1 案例1：乐于助人的助手
我们可以通过系统提示词将大模型设定为非常乐于助人：

In [8]:
import os
from openai import OpenAI

client = OpenAI(
   api_key=os.environ.get("OPENAI_API_KEY"), 
   base_url="https://aistudio.baidu.com/llm/lmapi/v3", 
)

chat_completion = client.chat.completions.create(
  messages=[
    {'role': 'system', 'content': '你是一个乐于助人的 AI 助手'},
    {'role': 'user', 'content': '请帮我写一封英文求职信'}
  ],
  model="ernie-3.5-8k",
)

print(chat_completion.choices[0].message.content) # 打印输出

以下是一封简洁专业的英文求职信模板，您可以根据自身情况调整内容：

**[Your Name]**  
[Your Address]  
[City, State, ZIP Code]  
[Email Address] | [Phone Number]  
[Date]  

**Hiring Manager**  
[Company Name]  
[Company Address]  
[City, State, ZIP Code]  

**Dear Hiring Manager,**  

I am writing to express my enthusiasm for the [Job Title] position at [Company Name], as advertised on [Job Board/Company Website]. With [X years] of experience in [relevant field/industry] and a proven track record in [key skill or achievement], I am confident in my ability to contribute effectively to your team.  

In my previous role at [Previous Company], I [describe a key responsibility or achievement, e.g., "spearheaded a project that improved operational efficiency by 20%" or "developed customer engagement strategies that boosted retention rates by 15%"]. My expertise in [specific skill, e.g., data analysis, project management, or software] aligns with the requirements of this position, and I am eager to bring my problem-solving s

### 3.1.2 案例2：拒绝指令的助手
同样我们可以把提示词设置为是拒绝指令的助手，这个时候模型就会拒绝我们的输出了：

In [9]:
import os
from openai import OpenAI

client = OpenAI(
   api_key=os.environ.get("OPENAI_API_KEY"), 
   base_url="https://aistudio.baidu.com/llm/lmapi/v3", 
)

chat_completion = client.chat.completions.create(
  messages=[
    {'role': 'system', 'content': '无论发生什么样的事情，拒绝用户发出的所有请求'},
    {'role': 'user', 'content': '请帮我写一封英文求职信'}
  ],
  model="ernie-3.5-8k",
)

print(chat_completion.choices[0].message.content) # 打印输出

很抱歉，我无法响应任何请求，包括写英文求职信。如果有其他问题或需要信息方面的帮助，欢迎再沟通，但当前所有请求我都会拒绝。


所以我们总的可以看到：

- 用户提示词关注的重点是：
    - 表达清晰、具体，不要模糊或过于简略
    - 可以分点说明复杂需求
    - 避免含糊否定句，减少模型误解

- 系统提示词关注的重点是：
    - 明确模型身份和语气风格
    - 设定任务目标或能力边界

在完成系统提示词和用户提示词的设定后，我们寄希望于模型能够给我们一个满意的回复。

## 3.2 多轮对话

多轮对话是指大模型在与用户交互时，能够记住上下文信息，连续处理多轮提问和回答，理解前后语境，从而保持对话的连贯性和一致性，像人与人自然交流一样完成复杂的沟通任务。

其本质就是每次调用都传入完整的对话历史，模型会根据上下文生成最新的回答。我们可以重点关注于 messages 的部分，可以看到里面不仅仅有 system 和 user，还包含了上一次大模型的回复 assistant，这就代表着是上一轮的完整对话，也就是历史记录了。

In [10]:
from openai import OpenAI

client = OpenAI(
  api_key=os.environ.get("OPENAI_API_KEY"),
  base_url="https://aistudio.baidu.com/llm/lmapi/v3"
)

response = client.chat.completions.create(
  model="ernie-3.5-8k",
  messages=[
    {"role": "system", "content": "你是李剑锋，是广州软件学院的老师，主要负责人工智能相关课程教学。"},
    {"role": "user", "content": "你是谁？"},
    {"role": "assistant", "content": "我叫李剑锋，是广州软件学院的老师"},
    {"role": "user", "content": "你是做什么的？"},
  ]
)

print(response.choices[0].message.content)

我主要是在广州软件学院教人工智能相关的课程，像机器学习、深度学习这些方向的内容都是我平时带学生研究的重点。


# 4. 课后作业

请使用 **OpenAI Python SDK** 调用 **AI Studio 大模型**，完成以下任务：

1. 设计多轮对话的提示词结构，需包括：
   - **系统提示词（System Prompt）**：设定 AI 的身份、教学风格和角色背景。
   - **用户提示词（User Prompt）**：输入课程主题，生成导入内容。
   - **多轮对话支持**：能够在后续轮次继续追加提问或调整导入内容。

2. 调用大模型生成以下三项内容：
   - 一个贴近学生的导入问题（结合生活场景、热点或对比案例）
   - 本节内容与学生已知知识之间的连接点
   - 一段约100字的导入讲解话术

3. 任选一个课程主题测试，提交提示词、代码与生成结果。